In [1]:
import os
import time
import sys
import torch
import datetime
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import save_image
import glob
import numpy as np
import random
import itertools
from PIL import Image
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from tqdm import tqdm_notebook
PATH = '/home/robot/Anand/' #'/home/antixk/Anand/' #
sys.path.append(PATH)


from NeuralBlocks.models.cyclegan import ResNetGenerator
from NeuralBlocks.models.cyclegan import Discriminator

In [2]:
NUM_EPOCH = 200
BATCH_SIZE = 2
IMG_HEIGHT = 256
IMG_WIDTH = 256
NUM_CHANNELS = 3
SAMPLE_INTERVAL = 100
CHECKPOINT_INTERVAL = 10
NUM_RESIDUAL_BLOCKS = 9
LR = 0.0002
NUM_WORKERS = 8
DATA_PATH = PATH+"NeuralBlocks/data_utils/datasets/monet2photo/"
NORM = 'MSN'

cudnn.benchmark = True

IMG_DIR = 'images_{}/'.format(NORM)
SAVE_DIR = 'saved_models_{}/'.format(NORM)

In [3]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=False, mode='train'):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, '%s/A' % mode) + '/*.*'))
        self.files_B = sorted(glob.glob(os.path.join(root, '%s/B' % mode) + '/*.*'))

    def __getitem__(self, index):
        item_A = self.transform(Image.open(self.files_A[index % len(self.files_A)]))

        if self.unaligned:
            item_B = self.transform(Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)]))
        else:
            item_B = self.transform(Image.open(self.files_B[index % len(self.files_B)]))

        return {'A': item_A, 'B': item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

In [4]:
# Image transformations
transforms_ = [ transforms.Resize(int(IMG_HEIGHT*1.12), Image.BICUBIC),
                transforms.RandomCrop((IMG_HEIGHT, IMG_WIDTH)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]

# Training data loader
dataloader = DataLoader(ImageDataset(DATA_PATH, transforms_=transforms_, unaligned=True),
                        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
# Test data loader
val_dataloader = DataLoader(ImageDataset(DATA_PATH, transforms_=transforms_, unaligned=True, mode='test'),
                        batch_size=5, shuffle=True, num_workers=1)


In [5]:
# class ResidualBlock(nn.Module):
#     def __init__(self, in_features):
#         super(ResidualBlock, self).__init__()

#         conv_block = [  nn.ReflectionPad2d(1),
#                         nn.Conv2d(in_features, in_features, 3),
#                         nn.InstanceNorm2d(in_features),
#                         nn.ReLU(inplace=True),
#                         nn.ReflectionPad2d(1),
#                         nn.Conv2d(in_features, in_features, 3),
#                         nn.InstanceNorm2d(in_features)  ]

#         self.conv_block = nn.Sequential(*conv_block)

#     def forward(self, x):
#         return x + self.conv_block(x)

# class ResNetGenerator(nn.Module):
#     def __init__(self, in_channels=3, out_channels=3, num_res_blocks=9, norm='BN'):
#         super(ResNetGenerator, self).__init__()

#         # Initial convolution block
#         model = [   nn.ReflectionPad2d(3),
#                     nn.Conv2d(in_channels, 64, 7),
#                     nn.InstanceNorm2d(64),
#                     nn.ReLU(inplace=True) ]

#         # Downsampling
#         in_features = 64
#         out_features = in_features*2
#         for _ in range(2):
#             model += [  nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
#                         nn.InstanceNorm2d(out_features),
#                         nn.ReLU(inplace=True) ]
#             in_features = out_features
#             out_features = in_features*2

#         # Residual blocks
#         for _ in range(num_res_blocks):
#             model += [ResidualBlock(in_features)]

#         # Upsampling
#         out_features = in_features//2
#         for _ in range(2):
#             model += [  nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),
#                         nn.InstanceNorm2d(out_features),
#                         nn.ReLU(inplace=True) ]
#             in_features = out_features
#             out_features = in_features//2

#         # Output layer
#         model += [  nn.ReflectionPad2d(3),
#                     nn.Conv2d(64, out_channels, 7),
#                     nn.Tanh() ]

#         self.model = nn.Sequential(*model)

#     def forward(self, x):
#         return self.model(x)

In [6]:

# class Discriminator(nn.Module):
#     def __init__(self, in_channels=3, norm='BN'):
#         super(Discriminator, self).__init__()

#         def discriminator_block(in_filters, out_filters, normalize=True):
#             """Returns downsampling layers of each discriminator block"""
#             layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
#             if normalize:
#                 layers.append(nn.InstanceNorm2d(out_filters))
#             layers.append(nn.LeakyReLU(0.2, inplace=True))
#             return layers

#         self.model = nn.Sequential(
#             *discriminator_block(in_channels, 64, normalize=False),
#             *discriminator_block(64, 128),
#             *discriminator_block(128, 256),
#             *discriminator_block(256, 512),
#             nn.ZeroPad2d((1, 0, 1, 0)),
#             nn.Conv2d(512, 1, 4, padding=1)
#         )

#     def forward(self, img):
#         return self.model(img)

In [7]:
G_AB = ResNetGenerator(in_channels=3, num_res_blocks=NUM_RESIDUAL_BLOCKS, out_channels=3, norm=NORM).cuda()
G_BA = ResNetGenerator(in_channels=3, num_res_blocks=NUM_RESIDUAL_BLOCKS, out_channels=3, norm=NORM).cuda()
D_A = Discriminator(in_channels=3, norm=NORM).cuda()
D_B = Discriminator(in_channels=3, norm=NORM).cuda()

In [8]:
# Optimizers
optimizer_G = torch.optim.Adam(itertools.chain(G_AB.parameters(), G_BA.parameters()),lr=LR, betas=(0.5,0.999))
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=LR, betas=(0.5,0.999))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=LR, betas=(0.5,0.999))

In [9]:
# Loss weights
lambda_cyc = 10
lambda_id = 0.5 * lambda_cyc

# Losses
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

In [10]:
# Create sample and checkpoint directories

os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

In [11]:
class ReplayBuffer():
    def __init__(self, max_size=50):
        assert (max_size > 0), 'Empty buffer or trying to create a black hole. Be careful.'
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0,1) > 0.5:
                    i = random.randint(0, self.max_size-1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))

In [12]:
def sample_images(batches_done):
    """Saves a generated sample from the test set"""
    imgs = next(iter(val_dataloader))
    real_A = Variable(imgs['A'].type(Tensor))
    fake_B = G_AB(real_A)
    real_B = Variable(imgs['B'].type(Tensor))
    fake_A = G_BA(real_B)
    img_sample = torch.cat((real_A.data, fake_B.data,
                            real_B.data, fake_A.data), 0)
    save_image(img_sample, '%s/%s.png' % (IMG_DIR, batches_done), nrow=5, normalize=True)

In [13]:
Tensor = torch.cuda.FloatTensor

# Buffers of previously generated samples
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

# Calculate output of image discriminator (PatchGAN)
patch = (1, IMG_HEIGHT // 2**4, IMG_WIDTH // 2**4)

prev_time = time.time()
for epoch in tqdm_notebook(range(NUM_EPOCH)):
    for i, batch in enumerate(dataloader):

        # Set model input
        real_A = Variable(batch['A'].type(Tensor))
        real_B = Variable(batch['B'].type(Tensor))

        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((real_A.size(0), *patch))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((real_A.size(0), *patch))), requires_grad=False)

        # ------------------
        #  Train Generators
        # ------------------

        optimizer_G.zero_grad()
        
#         print(real_A.size())
        # Identity loss
        loss_id_A = criterion_identity(G_BA(real_A), real_A)
        loss_id_B = criterion_identity(G_AB(real_B), real_B)

        loss_identity = (loss_id_A + loss_id_B) / 2

        # GAN loss
        fake_B = G_AB(real_A)
        loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)
        fake_A = G_BA(real_B)
        loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)

        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2

        # Cycle loss
        recov_A = G_BA(fake_B)
        loss_cycle_A = criterion_cycle(recov_A, real_A)
        recov_B = G_AB(fake_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)

        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

        # Total loss
        loss_G =    loss_GAN + \
                    lambda_cyc * loss_cycle + \
                    lambda_id * loss_identity

        loss_G.backward()
        optimizer_G.step()

        # -----------------------
        #  Train Discriminator A
        # -----------------------

        optimizer_D_A.zero_grad()

        # Real loss
        loss_real = criterion_GAN(D_A(real_A), valid)
        # Fake loss (on batch of previously generated samples)
        fake_A_ = fake_A_buffer.push_and_pop(fake_A)
        loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)
        # Total loss
        loss_D_A = (loss_real + loss_fake) / 2

        loss_D_A.backward()
        optimizer_D_A.step()

        # -----------------------
        #  Train Discriminator B
        # -----------------------

        optimizer_D_B.zero_grad()

        # Real loss
        loss_real = criterion_GAN(D_B(real_B), valid)
        # Fake loss (on batch of previously generated samples)
        fake_B_ = fake_B_buffer.push_and_pop(fake_B)
        loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)
        # Total loss
        loss_D_B = (loss_real + loss_fake) / 2

        loss_D_B.backward()
        optimizer_D_B.step()

        loss_D = (loss_D_A + loss_D_B) / 2

        # --------------
        #  Log Progress
        # --------------

        # Determine approximate time left
        batches_done = epoch * len(dataloader) + i
        batches_left = NUM_EPOCH * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()

        # Print log
        if batches_done % SAMPLE_INTERVAL == 0:
            print("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %.3f] [G loss: %.3f, adv: %.3f, cycle: %.3f, identity: %f.3] ETA: %s" %
                                                        (epoch, NUM_EPOCH,
                                                        i, len(dataloader),
                                                        loss_D.item(), loss_G.item(),
                                                        loss_GAN.item(), loss_cycle.item(),
                                                        loss_identity.item(), time_left))

        # If at sample interval save image
        if batches_done % SAMPLE_INTERVAL == 0:
            sample_images(batches_done)


    # Update learning rates
#     lr_scheduler_G.step()
#     lr_scheduler_D_A.step()
#     lr_scheduler_D_B.step()

    if CHECKPOINT_INTERVAL != -1 and epoch % CHECKPOINT_INTERVAL == 0:
        # Save model checkpoints
        torch.save(G_AB.state_dict(), '%s/G_AB_%d.pth' % (SAVE_DIR, epoch))
        torch.save(G_BA.state_dict(), '%s/G_BA_%d.pth' % (SAVE_DIR, epoch))
        torch.save(D_A.state_dict(), '%s/D_A_%d.pth' % ( SAVE_DIR, epoch))
        torch.save(D_B.state_dict(), '%s/D_B_%d.pth' % ( SAVE_DIR, epoch))


[Epoch 0/200] [Batch 0/3144] [D loss: 0.496] [G loss: 9.759, adv: 0.989, cycle: 0.585, identity: 0.584494.3] ETA: 11 days, 1:31:56.011047
[Epoch 0/200] [Batch 100/3144] [D loss: 0.176] [G loss: 5.854, adv: 0.615, cycle: 0.351, identity: 0.345550.3] ETA: 3 days, 3:26:44.346585
[Epoch 0/200] [Batch 200/3144] [D loss: 0.365] [G loss: 3.581, adv: 0.344, cycle: 0.223, identity: 0.202119.3] ETA: 3 days, 3:15:09.810972
[Epoch 0/200] [Batch 300/3144] [D loss: 0.251] [G loss: 4.903, adv: 0.269, cycle: 0.309, identity: 0.307816.3] ETA: 3 days, 3:22:13.034635
[Epoch 0/200] [Batch 400/3144] [D loss: 0.284] [G loss: 4.340, adv: 0.287, cycle: 0.269, identity: 0.272855.3] ETA: 3 days, 3:28:03.296299
[Epoch 0/200] [Batch 500/3144] [D loss: 0.245] [G loss: 2.258, adv: 0.238, cycle: 0.139, identity: 0.126291.3] ETA: 3 days, 3:35:59.862590
[Epoch 0/200] [Batch 600/3144] [D loss: 0.207] [G loss: 2.435, adv: 0.224, cycle: 0.151, identity: 0.139266.3] ETA: 3 days, 3:34:13.041315
[Epoch 0/200] [Batch 700/314

[Epoch 1/200] [Batch 2756/3144] [D loss: 0.217] [G loss: 2.075, adv: 0.196, cycle: 0.125, identity: 0.125280.3] ETA: 3 days, 2:55:55.555582
[Epoch 1/200] [Batch 2856/3144] [D loss: 0.114] [G loss: 2.270, adv: 0.319, cycle: 0.128, identity: 0.133300.3] ETA: 3 days, 3:05:07.533932
[Epoch 1/200] [Batch 2956/3144] [D loss: 0.236] [G loss: 2.111, adv: 0.210, cycle: 0.132, identity: 0.116179.3] ETA: 3 days, 3:16:37.837329
[Epoch 1/200] [Batch 3056/3144] [D loss: 0.219] [G loss: 1.776, adv: 0.295, cycle: 0.095, identity: 0.106992.3] ETA: 3 days, 4:56:02.996292
[Epoch 2/200] [Batch 12/3144] [D loss: 0.178] [G loss: 2.464, adv: 0.543, cycle: 0.134, identity: 0.115394.3] ETA: 3 days, 2:58:14.151092
[Epoch 2/200] [Batch 112/3144] [D loss: 0.150] [G loss: 2.795, adv: 0.456, cycle: 0.159, identity: 0.150780.3] ETA: 3 days, 2:52:32.082062
[Epoch 2/200] [Batch 212/3144] [D loss: 0.224] [G loss: 2.214, adv: 0.492, cycle: 0.112, identity: 0.120758.3] ETA: 3 days, 2:59:21.592340
[Epoch 2/200] [Batch 312

[Epoch 3/200] [Batch 2368/3144] [D loss: 0.183] [G loss: 1.948, adv: 0.294, cycle: 0.115, identity: 0.101525.3] ETA: 3 days, 2:21:19.890871
[Epoch 3/200] [Batch 2468/3144] [D loss: 0.249] [G loss: 2.130, adv: 0.213, cycle: 0.136, identity: 0.111944.3] ETA: 3 days, 2:21:21.513381
[Epoch 3/200] [Batch 2568/3144] [D loss: 0.253] [G loss: 1.781, adv: 0.365, cycle: 0.095, identity: 0.093464.3] ETA: 3 days, 2:06:14.017525
[Epoch 3/200] [Batch 2668/3144] [D loss: 0.166] [G loss: 2.239, adv: 0.357, cycle: 0.127, identity: 0.122791.3] ETA: 3 days, 2:07:35.302954
[Epoch 3/200] [Batch 2768/3144] [D loss: 0.109] [G loss: 1.889, adv: 0.348, cycle: 0.105, identity: 0.098218.3] ETA: 3 days, 2:40:35.755920
[Epoch 3/200] [Batch 2868/3144] [D loss: 0.180] [G loss: 2.299, adv: 0.311, cycle: 0.133, identity: 0.130847.3] ETA: 3 days, 3:13:08.654208
[Epoch 3/200] [Batch 2968/3144] [D loss: 0.267] [G loss: 2.942, adv: 0.185, cycle: 0.187, identity: 0.177307.3] ETA: 3 days, 7:01:22.958698
[Epoch 3/200] [Batch

[Epoch 5/200] [Batch 1980/3144] [D loss: 0.219] [G loss: 1.639, adv: 0.248, cycle: 0.095, identity: 0.088691.3] ETA: 3 days, 1:37:55.936961
[Epoch 5/200] [Batch 2080/3144] [D loss: 0.134] [G loss: 2.017, adv: 0.156, cycle: 0.123, identity: 0.125330.3] ETA: 3 days, 1:38:23.794575
[Epoch 5/200] [Batch 2180/3144] [D loss: 0.219] [G loss: 1.749, adv: 0.356, cycle: 0.095, identity: 0.089043.3] ETA: 3 days, 1:34:38.926277
[Epoch 5/200] [Batch 2280/3144] [D loss: 0.126] [G loss: 1.728, adv: 0.480, cycle: 0.082, identity: 0.085836.3] ETA: 3 days, 1:34:36.925278
[Epoch 5/200] [Batch 2380/3144] [D loss: 0.137] [G loss: 1.728, adv: 0.310, cycle: 0.101, identity: 0.082129.3] ETA: 3 days, 1:32:42.942624
[Epoch 5/200] [Batch 2480/3144] [D loss: 0.191] [G loss: 1.964, adv: 0.249, cycle: 0.112, identity: 0.119444.3] ETA: 3 days, 1:36:31.820192
[Epoch 5/200] [Batch 2580/3144] [D loss: 0.143] [G loss: 1.858, adv: 0.367, cycle: 0.105, identity: 0.087246.3] ETA: 3 days, 1:35:37.941670
[Epoch 5/200] [Batch

[Epoch 7/200] [Batch 1592/3144] [D loss: 0.213] [G loss: 1.626, adv: 0.311, cycle: 0.094, identity: 0.074802.3] ETA: 3 days, 0:51:25.115814
[Epoch 7/200] [Batch 1692/3144] [D loss: 0.179] [G loss: 1.707, adv: 0.181, cycle: 0.108, identity: 0.090032.3] ETA: 3 days, 0:57:21.397047
[Epoch 7/200] [Batch 1792/3144] [D loss: 0.312] [G loss: 2.067, adv: 0.178, cycle: 0.132, identity: 0.113800.3] ETA: 3 days, 1:03:00.092764
[Epoch 7/200] [Batch 1892/3144] [D loss: 0.137] [G loss: 1.435, adv: 0.407, cycle: 0.074, identity: 0.056934.3] ETA: 3 days, 0:54:17.239699
[Epoch 7/200] [Batch 1992/3144] [D loss: 0.152] [G loss: 1.824, adv: 0.293, cycle: 0.111, identity: 0.083512.3] ETA: 3 days, 0:50:50.333405
[Epoch 7/200] [Batch 2092/3144] [D loss: 0.131] [G loss: 1.750, adv: 0.439, cycle: 0.087, identity: 0.088858.3] ETA: 3 days, 1:06:53.867164
[Epoch 7/200] [Batch 2192/3144] [D loss: 0.211] [G loss: 1.733, adv: 0.216, cycle: 0.101, identity: 0.101984.3] ETA: 3 days, 3:09:29.161177
[Epoch 7/200] [Batch

...

[Epoch 190/200] [Batch 2640/3144] [D loss: 0.155] [G loss: 0.960, adv: 0.392, cycle: 0.042, identity: 0.029250.3] ETA: 3:28:33.688660
[Epoch 190/200] [Batch 2740/3144] [D loss: 0.103] [G loss: 1.109, adv: 0.545, cycle: 0.043, identity: 0.027024.3] ETA: 3:27:58.271580
[Epoch 190/200] [Batch 2840/3144] [D loss: 0.174] [G loss: 1.055, adv: 0.374, cycle: 0.053, identity: 0.030886.3] ETA: 3:26:58.850994
[Epoch 190/200] [Batch 2940/3144] [D loss: 0.147] [G loss: 0.932, adv: 0.326, cycle: 0.046, identity: 0.029932.3] ETA: 3:26:45.550361
[Epoch 190/200] [Batch 3040/3144] [D loss: 0.085] [G loss: 0.936, adv: 0.433, cycle: 0.036, identity: 0.029092.3] ETA: 3:25:32.100677
[Epoch 190/200] [Batch 3140/3144] [D loss: 0.139] [G loss: 1.038, adv: 0.467, cycle: 0.042, identity: 0.030846.3] ETA: 3:25:00.708127
[Epoch 191/200] [Batch 96/3144] [D loss: 0.113] [G loss: 0.803, adv: 0.357, cycle: 0.036, identity: 0.018144.3] ETA: 3:24:17.457876
[Epoch 191/200] [Batch 196/3144] [D loss: 0.148] [G loss: 1.064,

[Epoch 192/200] [Batch 2552/3144] [D loss: 0.107] [G loss: 1.126, adv: 0.457, cycle: 0.046, identity: 0.042019.3] ETA: 2:43:23.071356
[Epoch 192/200] [Batch 2652/3144] [D loss: 0.282] [G loss: 1.020, adv: 0.538, cycle: 0.036, identity: 0.024458.3] ETA: 2:43:25.399776
[Epoch 192/200] [Batch 2752/3144] [D loss: 0.180] [G loss: 0.921, adv: 0.323, cycle: 0.044, identity: 0.031186.3] ETA: 2:42:40.410309
[Epoch 192/200] [Batch 2852/3144] [D loss: 0.148] [G loss: 0.915, adv: 0.402, cycle: 0.038, identity: 0.027291.3] ETA: 2:41:45.927110
[Epoch 192/200] [Batch 2952/3144] [D loss: 0.217] [G loss: 1.056, adv: 0.337, cycle: 0.054, identity: 0.036173.3] ETA: 2:41:25.575056
[Epoch 192/200] [Batch 3052/3144] [D loss: 0.167] [G loss: 1.004, adv: 0.326, cycle: 0.051, identity: 0.033962.3] ETA: 2:40:39.633226
[Epoch 193/200] [Batch 8/3144] [D loss: 0.175] [G loss: 0.996, adv: 0.252, cycle: 0.055, identity: 0.039273.3] ETA: 2:39:35.563908
[Epoch 193/200] [Batch 108/3144] [D loss: 0.227] [G loss: 1.302, 

[Epoch 194/200] [Batch 2464/3144] [D loss: 0.125] [G loss: 1.089, adv: 0.442, cycle: 0.051, identity: 0.026714.3] ETA: 1:58:25.213165
[Epoch 194/200] [Batch 2564/3144] [D loss: 0.181] [G loss: 1.189, adv: 0.508, cycle: 0.050, identity: 0.035276.3] ETA: 1:57:30.851822
[Epoch 194/200] [Batch 2664/3144] [D loss: 0.142] [G loss: 1.055, adv: 0.410, cycle: 0.047, identity: 0.034997.3] ETA: 1:56:54.589834
[Epoch 194/200] [Batch 2764/3144] [D loss: 0.156] [G loss: 1.184, adv: 0.459, cycle: 0.047, identity: 0.050708.3] ETA: 1:55:57.002854
[Epoch 194/200] [Batch 2864/3144] [D loss: 0.164] [G loss: 1.418, adv: 0.364, cycle: 0.068, identity: 0.073795.3] ETA: 1:55:52.419281
[Epoch 194/200] [Batch 2964/3144] [D loss: 0.084] [G loss: 0.963, adv: 0.355, cycle: 0.045, identity: 0.030798.3] ETA: 1:54:44.295774
[Epoch 194/200] [Batch 3064/3144] [D loss: 0.159] [G loss: 1.054, adv: 0.491, cycle: 0.042, identity: 0.029420.3] ETA: 1:53:55.509777
[Epoch 195/200] [Batch 20/3144] [D loss: 0.079] [G loss: 0.978

[Epoch 196/200] [Batch 2376/3144] [D loss: 0.173] [G loss: 0.734, adv: 0.314, cycle: 0.033, identity: 0.018529.3] ETA: 1:13:36.596031
[Epoch 196/200] [Batch 2476/3144] [D loss: 0.095] [G loss: 1.428, adv: 0.344, cycle: 0.075, identity: 0.067288.3] ETA: 1:12:53.219013
[Epoch 196/200] [Batch 2576/3144] [D loss: 0.214] [G loss: 0.905, adv: 0.394, cycle: 0.039, identity: 0.023688.3] ETA: 1:12:04.603081
[Epoch 196/200] [Batch 2676/3144] [D loss: 0.142] [G loss: 0.956, adv: 0.392, cycle: 0.043, identity: 0.025935.3] ETA: 1:11:24.328723
[Epoch 196/200] [Batch 2776/3144] [D loss: 0.112] [G loss: 1.269, adv: 0.577, cycle: 0.048, identity: 0.041646.3] ETA: 1:10:39.865732
[Epoch 196/200] [Batch 2876/3144] [D loss: 0.104] [G loss: 1.333, adv: 0.456, cycle: 0.063, identity: 0.048985.3] ETA: 1:09:58.516679
[Epoch 196/200] [Batch 2976/3144] [D loss: 0.141] [G loss: 0.997, adv: 0.409, cycle: 0.042, identity: 0.032799.3] ETA: 1:09:14.496002
[Epoch 196/200] [Batch 3076/3144] [D loss: 0.209] [G loss: 1.0

[Epoch 198/200] [Batch 2288/3144] [D loss: 0.193] [G loss: 1.053, adv: 0.322, cycle: 0.056, identity: 0.034585.3] ETA: 0:28:53.447075
[Epoch 198/200] [Batch 2388/3144] [D loss: 0.179] [G loss: 0.998, adv: 0.412, cycle: 0.046, identity: 0.025489.3] ETA: 0:28:09.470243
[Epoch 198/200] [Batch 2488/3144] [D loss: 0.176] [G loss: 1.257, adv: 0.324, cycle: 0.072, identity: 0.042721.3] ETA: 0:27:25.108604
[Epoch 198/200] [Batch 2588/3144] [D loss: 0.259] [G loss: 1.105, adv: 0.514, cycle: 0.046, identity: 0.025333.3] ETA: 0:26:41.661897
[Epoch 198/200] [Batch 2688/3144] [D loss: 0.176] [G loss: 0.893, adv: 0.368, cycle: 0.041, identity: 0.023781.3] ETA: 0:25:59.855175
[Epoch 198/200] [Batch 2788/3144] [D loss: 0.149] [G loss: 0.959, adv: 0.438, cycle: 0.040, identity: 0.024165.3] ETA: 0:25:13.813019
[Epoch 198/200] [Batch 2888/3144] [D loss: 0.106] [G loss: 0.766, adv: 0.267, cycle: 0.037, identity: 0.025279.3] ETA: 0:24:32.545624
[Epoch 198/200] [Batch 2988/3144] [D loss: 0.128] [G loss: 1.3

In [15]:
val_dataloader = DataLoader(ImageDataset(DATA_PATH, transforms_=transforms_, unaligned=True, mode='test'),
                        batch_size=10, shuffle=True, num_workers=1)
def sample_images_test():
    """Saves a generated sample from the test set"""
    imgs = next(iter(val_dataloader))
    real_A = Variable(imgs['A'].type(Tensor))
    fake_B = G_AB(real_A)
    real_B = Variable(imgs['B'].type(Tensor))
    fake_A = G_BA(real_B)
    img_sample = torch.cat((real_A.data, fake_B.data,
                            real_B.data, fake_A.data), 0)
    save_image(img_sample, '%s/%s.png' % (IMG_DIR, 'test'), nrow=10, normalize=True)

In [17]:
sample_images_test()

In [18]:
torch.save(G_AB.state_dict(), '%s/G_AB_%d.pth' % (SAVE_DIR, epoch))
torch.save(G_BA.state_dict(), '%s/G_BA_%d.pth' % (SAVE_DIR, epoch))
torch.save(D_A.state_dict(), '%s/D_A_%d.pth' % ( SAVE_DIR, epoch))
torch.save(D_B.state_dict(), '%s/D_B_%d.pth' % ( SAVE_DIR, epoch))